# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [6]:
import pandas as pd
import numpy as np

In [7]:
from xml.etree import ElementTree as ET

In [8]:
document = ET.parse( './data/mondial_database.xml' )

In [9]:
root= document.getroot()


# Q1-10 countries with the lowest infant mortality rates

In [10]:
pd.DataFrame([[child.find('name').text, float(child.find('infant_mortality').text)] 
              for child in root.findall(".//infant_mortality/..")], columns=['country','infant mortality']).sort_values('infant mortality')[:10]


,country,infant mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


# Q2-2.10 cities with the largest population

In [12]:
pd.DataFrame([[child.find('name').text, int(child.find('population').text)] 
              for child in root.findall("./country/city/population/..")], columns=['city','population']).sort_values('population',ascending=False)[:10]

,city,population
165,Seoul,10229262
123,Hong Kong,7055071
154,Al Qahirah,6053000
75,Bangkok,5876000
87,Ho Chi Minh,3924435
166,Busan,3813814
205,New Taipei,3722082
84,Hanoi,3056146
153,Al Iskandariyah,2917000
204,Taipei,2626138


# Q3-3.10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [13]:
p = []
for child in root.findall('.//ethnicgroup/..//population/..'):
    p += [[i.text,float(i.get('percentage')),child.find('name').text,float(child.findall("population")[-1].text)] for i in child.findall('ethnicgroup')]
    


In [17]:
eth=pd.DataFrame(p,columns=['ethnic group','percentage','county','cpop'])
eth['epop']=eth['percentage']*eth['cpop']/100.0
eth[['ethnic group','epop']].groupby('ethnic group').sum().sort_values('epop',ascending=False)[:10]

,epop
ethnic group,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08


# Q4-name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [19]:
#creating car code dictionary
codedict={child.get('car_code'):child.find('name').text for child in root.findall('./country')}

In [20]:
#Create dataframe for rivers and its lengths
rivers=pd.DataFrame([[i.find('name').text,float(i.find('length').text),i.find('source').get('country')] for i in root.findall('./river/name/../length/..')], columns=['rname','length','scountry'])

In [23]:
#Max River and its length
maxriver=rivers.iloc[rivers['length'].idxmax('length')]
print('Longest river- '+maxriver['rname']+", Location- "+codedict[maxriver['scountry']])

Longest river- Amazonas, Location- Peru


In [24]:
#Largest Lake
lakes=pd.DataFrame([[i.find('name').text,float(i.find('area').text),i.find('located').get('country')] for i in root.findall('./lake/name/../area/../located/..')], columns=['lname','area','lco'])

In [27]:
maxlake=lakes.iloc[lakes['area'].idxmax()]
print('Largest lake- '+maxlake['lname']+", Location- "+codedict[maxlake['lco']])

Largest lake- Caspian Sea, Location- Russia


In [29]:
#Airport at highest elevation
airport=pd.DataFrame([[i.find('name').text,i.find('elevation').text,i.find('latitude').text,i.find('longitude').text] for i in root.findall('./airport/name/../latitude/../longitude/../elevation/..')], columns=['aname','elevation','latitude','longitude'])

In [31]:
airport['elevation']=airport['elevation'].astype(float)
maxairport=airport.iloc[airport['elevation'].idxmax()]
print('The highest Airport - '+maxairport['aname']+", Latitude-  "+maxairport['latitude']+", Longitude-  "+ maxairport['longitude'])

The highest Airport - El Alto Intl, Latitude-  -16.513339, Longitude-  -68.192256
